<a href="https://colab.research.google.com/github/jaegeral/notebooks/blob/main/Malspam_Analysis_csv_to_Timesketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-colab
!pip install -q pandas

# File import

In [ ]:
from io import StringIO
import pandas as pd

# I did parse the csv before with Google Sheets to get rid of some weirdness
filename = '/Downloads/bad_emails_INBOX.Archives.2020-09_toparse.csv'
df = pd.read_csv(filename, encoding="utf-8",parse_dates=True)
df.head()

## To pandas 

In [ ]:
df.head()

## Get "Date" to date

In [ ]:
df.info() 


In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)


In [ ]:
# convert the 'Date' column to datetime format 

df['Date']= pd.to_datetime(df['Date'], utc=True) 


In [ ]:
# change format
df['Date'] = pd.to_datetime(df["Date"].dt.strftime('%Y-%m-%dT%H:%M:%S'))


In [ ]:

df = df.rename(columns={"Date": "datetime"}) # rename to better timesketch import


In [ ]:
df.head()

# Timesketch

## Setup

In [ ]:
!pip install -q timesketch_api_client 
!pip install -q timesketch_import_client

from timesketch_api_client import client
from timesketch_import_client import importer

In [ ]:
SERVER_LOCATION = "http://127.0.0.1:5000"
USERNAME = "dev"
PASSWORD = "dev"

ts = client.TimesketchApi(SERVER_LOCATION, USERNAME, PASSWORD)
my_sketch = ts.get_sketch(1)

## Upload

In [ ]:
with importer.ImportStreamer() as streamer:
    streamer.set_sketch(my_sketch)
    streamer.set_timestamp_description('malicious attachments')
    streamer.set_timeline_name('malicious attachments')
    streamer.set_message_format_string(
        'Malicious attachment entry {Subject:s} {Attachment:s} {Malware:s}')

    streamer.add_data_frame(df)